In [39]:
from os import listdir
from sys import argv

import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import f1_score, roc_curve, roc_auc_score, precision_recall_curve

from sklearn.decomposition import PCA

from skimage import feature
from skimage.feature import greycomatrix
from skimage.feature import greycoprops
from scipy.stats import entropy
from skimage.feature import local_binary_pattern

from sklearn.svm import SVC
from skimage import measure



# Inicialização de alguma variáveis

In [40]:
argv = [3, "imgs", "gts"]

path_imgs = sorted([ argv[1] + '/' + i for i in listdir(argv[1]) ])
path_gts = sorted([ argv[2] + '/' + i for i in listdir(argv[2]) ])

n_images = len(path_imgs)

names_VI = ["ExG", "ExGR", "CIVE", "VEG", "WI", "NGRDI"]

In [41]:
GT = np.zeros((512*512*len(path_imgs)), dtype = int)
n_features = 4 * 6 * 1 # 4 posiçoes, 6 extratores, 1 VI
n_levels = 64
VI = [ np.zeros((32*32*len(path_imgs), n_features), dtype = float) for i in range(6) ]
bic = np.zeros((32*32*len(path_imgs), n_levels * 2), dtype = int)

# Declaração de funções (e alguns testes)

In [42]:
def neighbMax(image):
	O = np.zeros(image.shape, dtype = float)
	ns = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    

	for lin in range(512):
		for col in range(512):
			_list = []
			for n in ns:
				if lin + n[0] >= 0 and lin + n[0] < 512 and col + n[1] >= 0 and col + n[1] < 512:
					_list.append( image[ lin + n[0] ] [col + n[1] ] )
			O[lin][col] = np.average(sorted(_list, reverse = True)[:3]) # Pega os 3 maiores

	return O

In [43]:
def neighbMin(image):
	O = np.zeros(image.shape, dtype = float)
	ns = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    

	for lin in range(512):
		for col in range(512):
			_list = []
			for n in ns:
				if lin + n[0] >= 0 and lin + n[0] < 512 and col + n[1] >= 0 and col + n[1] < 512:
					_list.append( image[ lin + n[0] ] [col + n[1] ] )
			O[lin][col] = np.average(sorted(_list, reverse = False)[:3]) # Pega os 3 menores

	return O

In [44]:
Z = np.random.randint(0, 255, (4,4))
I = cv2.normalize(Z, I, 0.0, 7.0, cv2.NORM_MINMAX, dtype = 1)
print(I.dtype)

int8


In [45]:
def BIC(image, levels):
    """
    beiradas da imagem devem ser consideradas o que?
    """
    interior = np.zeros(levels, dtype = int)
    borda = np.zeros(levels, dtype = int)
    ns = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    
    lim_lin = image.shape[0]
    lim_col = image.shape[1]

    for lin in range(image.shape[0]):
        for col in range(image.shape[1]): # laço do elementos da matriz
            curr = image[lin, col]
            interior[curr] += 1
            for n in ns: # laço dos vizinhos
                if lin + n[0] >= 0 and lin + n[0] < lim_lin and col + n[1] >= 0 and col + n[1] < lim_col: # teste de estar dentro da matriz
                    if image[lin + n[0], col + n[1]] != curr:
                        borda[curr] += 1
                        interior[curr] -=1
                        break
                    

    return interior, borda

In [31]:
BIC(M, 2) # 0, 1 e 16, 8

(array([0, 4]), array([ 5, 16]))

In [136]:
m = ["default", "ror", "uniform", "var"]

In [2]:
M = np.array([[1, 1 , 1, 1, 1], [1 , 0 , 0 , 0, 1], [1, 0 , 1, 1,  1], [1, 0 , 1, 1, 1], [1, 1, 1, 1, 1]])
print(M)

[[1 1 1 1 1]
 [1 0 0 0 1]
 [1 0 1 1 1]
 [1 0 1 1 1]
 [1 1 1 1 1]]


In [46]:
def entropy_func(P):
    return [entropy(P[:, :, 0, i].ravel()) for i in range(P.shape[3])]

In [47]:
def maxProb(P):
    return [np.max(P[:, :, 0, i]) for i in range(P.shape[3])]

In [48]:
def get_features(P):
    contrast = greycoprops(P, prop = "contrast")[0 , :]
    correlation = greycoprops(P, prop = "correlation")[0 , :]
    energy = greycoprops(P, prop = "ASM")[0 , :]
    homogeneity = greycoprops(P, prop = "homogeneity")[0 , :]
    _maxProb = maxProb(P)
    _entropy = entropy_func(P)
    
    #print(contrast, correlation, energy, homogeneity, _maxProb, _entropy)

    return [*contrast, *correlation, *energy, *homogeneity, *_maxProb, *_entropy]

In [54]:
for i in range(len(path_imgs)):

    
    img = cv2.imread(path_imgs[i], 1)
    gt = cv2.imread(path_gts[i], 0)
    gt = cv2.normalize(gt, gt, 0.0, 1.0, cv2.NORM_MINMAX)
    #GT = np.concatenate((GT, gt.ravel()), axis = 0)
    print(i)
    #print(path_imgs[i])
    
    ims = 512*512
    bls = 16*16
    nbl = 512/16
    
    _type = 1

    for lin in range(512)[::16]:
        for col in range(512)[::16]:
            offset = ims*i + (lin*512+col*16)
            
            GT[ offset : offset + 16*16 ] = gt[lin:lin+16, col:col+16].ravel()

    #B, G, R = cv2.split(img) # b = img[:,:,0]
    B, G, R = [np.float32(img[:, :, c]) for c in range(3)]
    r = R / (R + G + B)
    g = G / (R + G + B)
    b = B / (R + G + B)

    I = np.zeros(img.shape[:2], dtype = float)

    j = 0

    # ExG
    #print("ExG")
    Z = 2 * g - r - b
    I = cv2.normalize(Z, I, 0.0, 7.0, cv2.NORM_MINMAX, dtype = _type) # Max = 7?
    
    #I = neighbMax(I)
    b = 0
    for lin in range(512)[::16]:
        for col in range(512)[::16]:
            P = greycomatrix(I[lin:lin+16, col:col+16], [1], (np.pi/4, 0, 3*np.pi/2, 7*np.pi/4), 8)
            VI[j][b, :] = get_features(P)
            b += 1
    j+=1


    # ExGR
    #print("ExGR")
    Z = Z - 1.4 * r - g
    I = cv2.normalize(Z, I, 0.0, 7.0, cv2.NORM_MINMAX, dtype = _type) # Max = 7?
    #I = neighbMax(I)
    b = 0
    for lin in range(512)[::16]:
        for col in range(512)[::16]:
            P = greycomatrix(I[lin:lin+16, col:col+16], [1], (np.pi/4, 0, 3*np.pi/2, 7*np.pi/4), 8)            
            VI[j][b, :] = get_features(P)            
            b += 1
    j+=1

    # CIVE
    #print("CIVE")
    Z = 0.441 * r - 0.881 * g + 0.385 * b + 18.78745
    I = 7 - cv2.normalize(Z, I, 0.0, 7.0, cv2.NORM_MINMAX, dtype = _type) # Max = 7?
    #I = neighbMax(I)
    b = 0
    for lin in range(512)[::16]:
        for col in range(512)[::16]:
            P = greycomatrix(I[lin:lin+16, col:col+16], [1], (np.pi/4, 0, 3*np.pi/2, 7*np.pi/4), 8)
            VI[j][b, :] = get_features(P)
            b += 1
    j+=1

    # VEG
    #print("VEG")
    Z = g / (2 + r ** 0.667 * b ** (1 - 0.667))
    I = cv2.normalize(Z, I, 0.0, 7.0, cv2.NORM_MINMAX, dtype = _type) # Max = 7?
    #I = neighbMax(I)
    b = 0
    for lin in range(512)[::16]:
        for col in range(512)[::16]:
            P = greycomatrix(I[lin:lin+16, col:col+16], [1], (np.pi/4, 0, 3*np.pi/2, 7*np.pi/4), 8)
            VI[j][b, :] = get_features(P)             
            b += 1
    j+=1

    # WI
    #print("WI")
    Z = (g - b) / (r - g + 255)
    I = cv2.normalize(Z, I, 0.0, 7.0, cv2.NORM_MINMAX, dtype = _type) # Max = 7?
    #I = neighbMax(I)
    b = 0
    for lin in range(512)[::16]:
        for col in range(512)[::16]:
            P = greycomatrix(I[lin:lin+16, col:col+16], [1], (np.pi/4, 0, 3*np.pi/2, 7*np.pi/4), 8)
            VI[j][b, :] = get_features(P) 
            b += 1
    j+=1

    # NGRDI
    #print("NGRDI")
    Z = (G/2 - R/2) / (G/2 + R/2)
    I = cv2.normalize(Z, I, 0.0, 7.0, cv2.NORM_MINMAX, dtype = _type) # Max = 7?
    #I = neighbMax(I)
    b = 0
    for lin in range(512)[::16]:
        for col in range(512)[::16]:
            P = greycomatrix(I[lin:lin+16, col:col+16], [1], (np.pi/4, 0, 3*np.pi/2, 7*np.pi/4), 8)
            VI[j][b, :] = get_features(P) 
            b += 1
    j+=1
    
    
    #BIC
    bic_img = np.zeros(B.shape, dtype = int)
    for c in range(3):
        #print(img[:, :, c].shape, bic_img.shape, I.shape)
        bic_img += cv2.normalize(img[:, :, c], I, 0.0, 3.0, cv2.NORM_MINMAX, dtype = _type) * (4**c)
    
    b = 0
    for lin in range(512)[::16]:
        for col in range(512)[::16]:
            bic[b, 0:64], bic[b, 64:64*2]  = BIC(bic_img[lin:lin+16, col:col+16], 64)
            b += 1
            
    
print("DONE.")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


# Análise das VIs (curva ROC)

In [14]:
fpr = [np.array([0]) for i in range(6)]
tpr = [np.array([0]) for i in range(6)]

In [17]:
for j in range(6):
    print("xxx", "AUC =", roc_auc_score(GT[::256], VI[1][: , j]))
    #fpr[j], tpr[j], _ = roc_curve(GT, VI[j])

xxx AUC = 0.521750973875
xxx AUC = 0.521778998056
xxx AUC = 0.521782026383
xxx AUC = 0.521791195362
xxx AUC = 0.521162143328
xxx AUC = 0.521139587818


# Salvar VIs

In [ ]:
VIs_save_dir = "VI_2/MiniNBmax/"

In [ ]:
for j in range(6):
    np.save(VIs_save_dir + names_VI[j], VI[j])

In [ ]:
GT_save_dir = "VI_2/Mini/GT"

In [ ]:
np.save(GT_save_dir , GT)

In [ ]:
X = np.zeros((512*512*len(path_imgs), 6), dtype = float)

# Gerar e salvar PCA das VIs

In [ ]:
for i in range(6):
    X[:, i] = np.load("VI_2/MiniNBmax/" + names_VI[i] + ".npy")

In [ ]:
print(np.count_nonzero(X))

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

In [ ]:
print(np.count_nonzero(X_pca))

In [ ]:
np.save(VIs_save_dir + "PCA0", X_pca[:,0])
np.save(VIs_save_dir = "PCA1", X_pca[:,1])

# SVC

In [55]:
X = np.zeros((32*32*n_images, n_features * 6 + n_levels * 2), dtype = float)

In [56]:
for j in range(6):
    X[:, j * 24 : (j + 1) * 24] = VI[j]
X[:, n_features * 6 : n_features * 6 + n_levels*2] = bic

In [57]:
X.shape

(22528, 272)

In [58]:
X = np.column_stack((X, GT[::256]))

In [59]:
print(X.shape)
print(np.count_nonzero(X))

(22528, 273)
161793


In [60]:
np.random.shuffle(X)

In [61]:
GT = X[:,-1]
X = X[:,:-1]
print(X.shape)
print(np.count_nonzero(X))
print(GT.shape)
print(np.count_nonzero(GT))

(22528, 272)
154241
(22528,)
7552


In [62]:
X_train, X_test = np.split(X, [32*32*16], axis = 0)
y_train, y_test = np.split(GT, [32*32*16])

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(np.count_nonzero(X_train))
print(np.count_nonzero(y_train))
print(np.count_nonzero(X_test))
print(np.count_nonzero(y_test))

(16384, 272)
(16384,)
(6144, 272)
(6144,)
107317
5476
46924
2076


In [63]:
clf = SVC(max_iter = -1, probability=True)
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [64]:
predict = clf.predict_proba(X_test)[:,1]

In [65]:
print("xxx", "AUC =", roc_auc_score(y_test, predict))

xxx AUC = 0.524945862533
